Use dict to record every author's ID, name, publishment and citation

In [ ]:
import pandas as pd

def find_authors_id(s:str):
    return list(map(int, s.split(';')))

def find_authors_full_names(s:str):
    return s.split(';')

def find_affiliations(s:str):
    s = s.split(';')
    afs = []
    for a in s:
        parts = a.split(',')
        afs.append(','.join(parts[1:]))
    return afs


df = pd.read_csv("data from scopus.csv")

for index, row in df.iterrows(): # NOTE: Replace NA with 0
    df.loc[index,'Cited by'] = 0 if pd.isna(row['Cited by']) else row['Cited by']

authors = dict()

missing = 0

for index, row in df.iterrows():
    author_ids_str = str(row["Author(s) ID"])
    author_full_names_str = row["Author full names"]
    author_with_aff = row["Authors with affiliations"]

    if isinstance(author_full_names_str, float):
        # print(row['Title'])
        missing += 1
        continue
    if isinstance(row["Authors with affiliations"], float):
        continue

    aff = find_affiliations(author_with_aff)
    # print(aff)

    author_ids = find_authors_id(author_ids_str)
    author_full_names = find_authors_full_names(author_full_names_str)
    for k in range(len(author_ids)):
        id = author_ids[k]
        if id not in authors:
            authors[id] = dict()
            authors[id]['full_names'] = author_full_names[k].split('(')[0].strip()
            authors[id]['publication'] = 1
            authors[id]['cited'] = row["Cited by"]
            authors[id]['affiliations'] = set()
            authors[id]['affiliations'].add(aff[k])

        else:
            authors[id]['publication'] += 1
            authors[id]['cited'] +=  row["Cited by"]
            authors[id]['affiliations'].add(aff[k])


headers = ["ID", "full_names", "publication", "cited", "average_cited", "affiliations"] 

rows = []  


for key, value in authors.items():  

    rows.append([key,value['full_names'],value['publication'],value['cited'],value['cited']/value['publication'],';'.join(value['affiliations'])])  

 
df2 = pd.DataFrame(rows, columns=headers)  

df2.to_excel('information_of_authors.xlsx')

print("Papers without author's information:", missing) # 107


Build some dataframes with additional columns including information of the countries, constitutes, authors of our interest.

In [ ]:
df1 = pd.read_excel('top10_productive_countries.xlsx')  
top10_countries = list(df1['Country'])

df2 = pd.read_excel('top10_productive_institutes.xlsx')  
top10_institues = list(df2['Institutes'])

df3= pd.read_excel('top10_productive_authors.xlsx') 
top10_productive_authors = list(map(str,list(df3['Author_ID'])))

df4 = pd.read_excel('top10_highly_cited_authors.xlsx')  
top10_highly_cited_authors = list(map(str,list(df4['Author_ID'])))

for c in top10_countries:
    df[c] = 0

for i in top10_institues:
    df[i] = 0

for pa in top10_productive_authors:
    df[pa] = 0

for ca in top10_highly_cited_authors:
    df[ca] = 0

df['SCP'] = 0
df['MCP'] = 0


def find_countries(s:str):
    s = s.split(';')
    countries = [a.split(',')[-1] for a in s]
    countries = set([c.strip() for c in countries])
    return countries

missing = 0

for index, row in df.iterrows():
    affiliations = row["Affiliations"]

    if isinstance(row["Affiliations"],float): # NA
        missing += 1
        continue

    countries = find_countries(affiliations)
    for c in top10_countries: 
        if c in countries:
            df.loc[index,c] = 1
    if "Taiwan" in countries or "Hong Kong" in countries or "Macao" in countries:
        df.loc[index,"China"] = 1


    if len(countries) == 1:
        df.loc[index,'SCP'] = 1
    elif len(countries) > 1:
        # print(countries)
        China_all = ["China", "Taiwan", "Hong Kong", "Macao"]
        countries_tmp = [x for x in countries if x not in China_all] 
        if len(countries_tmp) == 0:
            df.loc[index,'SCP'] = 1 
        else:
            df.loc[index,'MCP'] = 1


    for inst in top10_institues: 
        if inst in affiliations:
            df.loc[index,inst] = 1

print("Papers without information of constitutes：",missing)


missing = 0

for index, row in df.iterrows():

    author_ids_str = str(row["Author(s) ID"])

    if isinstance(row["Author(s) ID"], float): # NA
        missing += 1
        continue

    for pa in top10_productive_authors:
        if pa in author_ids_str:
            df.loc[index,pa] = 1
    
    for ca in top10_highly_cited_authors:
        if ca in author_ids_str:
            df.loc[index,ca] = 1




# print("Papers without author's information：", missing)

df.to_excel("data_augmented.xlsx")
 

Calculate the publishment, SCP, MCP, total citations, H-index of all countries

In [3]:
def H_index(citations):  
    """calculate H-index"""
    citations.sort(reverse=True)  
    h = 0  
    for i, citation in enumerate(citations):  
        if i + 1 > citation:  # the index generated by *enumerate* function begins from 0，but H-index begins from 1 
            break  
        h = i + 1  
    return h  

for index, row in df1.iterrows():
    c = row['Country']
    df1.loc[index,'Counts'] = df[c].sum() 
    df1.loc[index,'SCP'] = df.loc[df[c] == 1, 'SCP'].sum() 
    df1.loc[index,'MCP'] = df.loc[df[c] == 1, 'MCP'].sum()
    df1.loc[index,'Total citations'] = df.loc[df[c] == 1, "Cited by"].sum()
    df1.loc[index,'Average article citations'] = \
        df1.loc[index,'Total citations'] / df1.loc[index,'Counts']
    df1.loc[index,'H-index'] = H_index(list(df.loc[df[c] == 1, "Cited by"]))

df1.to_excel("top10_productive_countries.xlsx")


Calculate the publishment, total reference, average reference and H-index of every constitutes

In [4]:
for index, row in df2.iterrows():
    inst = row['Institutes']
    df2.loc[index,"Counts"] = df[inst].sum()
    df2.loc[index,'Total citations'] = df.loc[df[inst] == 1, "Cited by"].sum()
    df2.loc[index,'Average article citations'] = \
        df2.loc[index,'Total citations'] / df2.loc[index,"Counts"] 
    df2.loc[index,'H-index'] = H_index(list(df.loc[df[inst] == 1, "Cited by"]))

df2.to_excel("top10_productive_institutes.xlsx")

Calculate the top10 productive authors' H-index

In [5]:
for index, row in df3.iterrows():
    author_id = str(row["Author_ID"])
    df3.loc[index,'H-index'] = H_index(list(df.loc[df[author_id] == 1, 'Cited by']))

df3.to_excel("top10_productive_authors.xlsx")